<a href="https://colab.research.google.com/github/Fatai-soliu/Text-Classification/blob/main/Soliu_Fatai_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.15.0
Eager mode:  True
Hub version:  0.16.1
GPU is NOT AVAILABLE


In [11]:
url = 'https://github.com/Fatai-soliu/Text-Classification/raw/main/Scopusdataset.xlsx'

papers = pd.read_excel(url, engine='openpyxl')

In [12]:
# View first 5 rows of data

papers.head()

,Authors,Title,Year,Source title,Link,Abstract,Author Keywords,Subject
0,Belguith S.; Kaaniche N.; Laurent M.; Jemai A....,Accountable privacy preserving attribute based...,2020,Journal of Parallel and Distributed Computing,https://www.scopus.com/inward/record.uri?eid=2...,"In this paper, we propose an accountable priva...",Accountability; Attribute based encryption; At...,Mathematics
1,Kumaran G.; Sivaraj R.; Ramachandra Prasad V.;...,Finite difference computation of free magneto-...,2021,Physica Scripta,https://www.scopus.com/inward/record.uri?eid=2...,It is essential to account the variability in ...,Chemical reaction; Keller box method; Powell-e...,Mathematics
2,Zahedi E.; Saraee M.; Masoumi F.S.; Yazdinejad M.,Regularized Contrastive Masked Autoencoder Mod...,2023,Algorithms,https://www.scopus.com/inward/record.uri?eid=2...,"Unsupervised anomalous sound detection, especi...",anomalous sound detection; Contrastive learnin...,Mathematics
3,Oladipupo M.A.; Obuzor P.C.; Bamgbade B.J.; Ol...,An Automated Python Script for Data Cleaning a...,2023,Informatica (Slovenia),https://www.scopus.com/inward/record.uri?eid=2...,Every employee in the company who deals with d...,automation; data cleaning; financial dataset; ...,Mathematics
4,Roy A.K.; Debnath S.; Bég O.A.,Transient solute dispersion in electro-osmotic...,2023,ZAMM Zeitschrift fur Angewandte Mathematik und...,https://www.scopus.com/inward/record.uri?eid=2...,The transport of a neutral solute in incompres...,NaN,Mathematics


In [13]:
# Remove columns that does not relate to the task

papers = papers.drop(columns=['Authors','Source title','Link', 'Author Keywords', 'Title', 'Year'], axis=1)

#print out the first five rows of papers
papers.head()

,Abstract,Subject
0,"In this paper, we propose an accountable priva...",Mathematics
1,It is essential to account the variability in ...,Mathematics
2,"Unsupervised anomalous sound detection, especi...",Mathematics
3,Every employee in the company who deals with d...,Mathematics
4,The transport of a neutral solute in incompres...,Mathematics


In [14]:
#Assign Label

def assign_label(subject):
    if subject == "Arts and Humanity":
        return 0
    elif subject == "Business management":
        return 1
    elif subject == "Computer science and engineering":
        return 2
    elif subject == "Mathematics":
        return 3
    elif subject == "Medicine":
        return 4
    else:
        return None  # Handle other cases if needed

# Apply the function to your DataFrame
papers["label"] = papers["Subject"].apply(assign_label)



papers.tail()

,Abstract,Subject,label
1137,Historically the British Trades Union Congress...,Arts and Humanity,0
1138,‘Emotional Weather Report’ is a song by Tom Wa...,Arts and Humanity,0
1139,Even though the rise of the neural machine tra...,Arts and Humanity,0
1140,This article responds to McCloud’s theoretical...,Arts and Humanity,0
1141,Background and objectives: Visual Schema Displ...,Arts and Humanity,0


In [15]:
# Drop Subject column

papers = papers.drop(columns=['Subject'], axis=1)

#print out the first five rows of papers
papers.head()

,Abstract,label
0,"In this paper, we propose an accountable priva...",3
1,It is essential to account the variability in ...,3
2,"Unsupervised anomalous sound detection, especi...",3
3,Every employee in the company who deals with d...,3
4,The transport of a neutral solute in incompres...,3


In [16]:
# Check if there is any missing data

papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1142 entries, 0 to 1141
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Abstract  1142 non-null   object
 1   label     1142 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 18.0+ KB


In [17]:
#Prepare data for model training
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True